In [2]:
# 예: 학습 완료 모델 로드
encoder = torch.load("/Data/nh25/ECG/nh_submission3/model/phase1_encoder_epoch10.pt")  # state_dict가 아니라 객체로 저장했을 경우

# alpha 값 확인
alpha_values = encoder.lead_gate.alpha.detach().cpu().numpy().squeeze()
for i, val in enumerate(alpha_values, start=1):
    print(f"Lead {i}: {val:.4f}")


AttributeError: 'dict' object has no attribute 'lead_gate'

In [1]:
# 구조 먼저 만들고 state_dict 로드
from model import *
from util_nh import *

encoder = EnhancedECGEncoder(in_channels=12)
encoder.load_state_dict(torch.load("/Data/nh25/ECG/nh_submission3/model/phase1_encoder_epoch10.pt"))

# alpha 확인
alpha_values = encoder.lead_gate.alpha.detach().cpu().numpy().squeeze()
print(alpha_values)


RuntimeError: Error(s) in loading state_dict for EnhancedECGEncoder:
	Missing key(s) in state_dict: "lead_gate.alpha", "stem.0.conv_small.weight", "stem.0.conv_small.bias", "stem.0.conv_medium.weight", "stem.0.conv_medium.bias", "stem.0.conv_large.weight", "stem.0.conv_large.bias", "stem.0.bn.weight", "stem.0.bn.bias", "stem.0.bn.running_mean", "stem.0.bn.running_var", "layer1.0.0.weight", "layer1.0.1.weight", "layer1.0.1.bias", "layer1.0.1.running_mean", "layer1.0.1.running_var", "layer1.0.3.weight", "layer1.0.4.weight", "layer1.0.4.bias", "layer1.0.4.running_mean", "layer1.0.4.running_var", "layer1.1.0.weight", "layer1.1.1.weight", "layer1.1.1.bias", "layer1.1.1.running_mean", "layer1.1.1.running_var", "layer1.1.3.weight", "layer1.1.4.weight", "layer1.1.4.bias", "layer1.1.4.running_mean", "layer1.1.4.running_var", "tatt1.conv.weight", "tatt1.conv.bias", "layer2.0.0.weight", "layer2.0.1.weight", "layer2.0.1.bias", "layer2.0.1.running_mean", "layer2.0.1.running_var", "layer2.0.3.weight

In [2]:
import torch

def load_encoder_and_get_alpha(ckpt_path, in_channels=12, device="cpu"):
    # 1) 체크포인트 로드
    ckpt = torch.load(ckpt_path, map_location=device)

    # 2) 실제 state_dict 꺼내기
    if isinstance(ckpt, dict) and "state_dict" in ckpt:
        sd = ckpt["state_dict"]
    elif isinstance(ckpt, dict) and "encoder" in ckpt and isinstance(ckpt["encoder"], dict):
        sd = ckpt["encoder"]
    else:
        sd = ckpt  # 이미 순수 state_dict인 경우

    # 3) prefix 정리
    new_sd = {}
    for k, v in sd.items():
        if k.startswith("module."):
            k = k[len("module."):]
        if k.startswith("encoder."):
            k = k[len("encoder."):]
        new_sd[k] = v

    # 4) 현재 코드의 모델 인스턴스 생성
    enc = EnhancedECGEncoder(in_channels=in_channels).to(device)

    # 5) state_dict 로드 (strict=False로 아키텍처 차이를 견딤)
    incompatible = enc.load_state_dict(new_sd, strict=False)

    # 6) alpha 존재 여부 확인
    alpha_ok = hasattr(enc, "lead_gate") and hasattr(enc.lead_gate, "alpha")
    alpha_tensor = enc.lead_gate.alpha.detach().cpu() if alpha_ok else None

    return enc, incompatible, alpha_ok, alpha_tensor

# 사용 예시
enc, inc, alpha_ok, alpha = load_encoder_and_get_alpha(
    "/Data/nh25/ECG/nh_submission3/model/phase1_encoder_epoch10.pt",
    in_channels=12, device="cpu"
)

print("Missing:", inc.missing_keys)
print("Unexpected:", inc.unexpected_keys)
print("Has alpha?:", alpha_ok)

if alpha_ok:
    print("alpha:", alpha.numpy().squeeze())


Missing: []
Unexpected: []
Has alpha?: True
alpha: [1.3222833  1.5937221  0.6384284  1.6823562  0.8638819  1.147478
 0.16227692 0.1362344  0.1767093  0.22961861 0.42116055 0.34676102]


NameError: name 'alpha_values' is not defined